In [2]:
import numpy as np
import cv2

In [7]:
def show_image(img):
    WINDOW_NAME = 'image'
    cv2.namedWindow(WINDOW_NAME)
    cv2.startWindowThread()
    cv2.imshow(WINDOW_NAME, img)
    cv2.waitKey(10000)
    cv2.destroyAllWindows()

In [31]:
#Load the image
path_to_image = '../../../../holes.jpg'
holes = cv2.imread(path_to_image)
path_to_image_islands = '../../../../islands.jpg'
islands = cv2.imread(path_to_image_islands)
path_to_image_example = '../../../../binary.png'
example = cv2.imread(path_to_image_example)
show_image(holes)
show_image(islands)
show_image(example)

In [46]:
#Binarize according to threshold
threshold = 127
ret, binarized = cv2.threshold(example, threshold, 255, cv2.THRESH_BINARY)
binarized = binarized[:,:,0]
show_image(binarized)

In [81]:
#fill image using contours
#The option RETR_CCOMP finds two level contours, the top level are those of the CCs
filled = binarized.copy()
img, contours, hierarchy = cv2.findContours(filled,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
    cv2.drawContours(filled, [cnt], 0, 255, -1)
show_image(filled)

In [80]:
#Get the holes by substracting the filled image
all_the_holes = cv2.bitwise_and(filled, cv2.bitwise_not(binarized))
holes_opened = cv2.morphologyEx(all_the_holes, cv2.MORPH_OPEN, np.ones((4,4),np.uint8))
show_image(all_the_holes)
show_image(holes_opened)

In [12]:
#Find the connected components (similar to bwlabel)
retval, labels = cv2.connectedComponents(binarized[:,:,0])

In [26]:
#An ugly transformation to see the CC's in different colors
dim = np.ceil(np.power(retval, 1.0/3))
step = int(np.floor(255/dim))
print dim, step
rvalues = np.arange(int(np.floor(step/2)),256, step)
colormapping = dict([(i,[rvalues[np.mod(i, dim)], 
                         rvalues[np.floor(np.mod(i, dim*dim)/dim)], 
                         rvalues[np.floor(i/dim)]]) for i in xrange(1, retval)])
colormapping[0] = [0,0,0]
colored_labels = np.array([[colormapping[i] for i in x]  for x in labels], dtype='uint8')
show_image(colored_labels)

2.0 127


/home/dafne/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
